In [2]:
# compute DTW distances
from dtaidistance import dtw
import pandas as pd
import numpy as np
#from py3plex.core import multinet
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt

import igraph as ig
import leidenalg as la

/tmp/ipykernel_2685763/1358168717.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


# build multiplex network for Fetal Liver group

In [25]:
# loading gene expression time series
ExprPath = "/home/csj/mywork/221107-path/221112-FL-ILCprimed-PathExpr.csv"
time_series = pd.read_csv(ExprPath, index_col=0)

# loading scenic network modules
scenic_path='/home/csj/mywork/TSC/fl_regulons_seed1.csv' #'./fl_regulons_seed1.csv'
reg = pd.read_csv(scenic_path,
                  index_col=[0,1], header=[0,1])\
      .droplevel(0,axis=1).reset_index()


In [26]:
# quality control, filter out time series with low accumulations
time_series = time_series[time_series.sum(axis=1)>0.2]
time_series.to_csv("FL-time_series.csv")

In [27]:
reg

,TF,MotifID,AUC,NES,MotifSimilarityQvalue,OrthologousIdentity,Annotation,Context,TargetGenes,RankAtMax
0,Ar,transfac_pro__M08908,0.073686,3.037615,0.000000,0.877642,gene is orthologous to ENSG00000169083 in H. s...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Ar', 1.0), ('Rdh12', 0.4101501149570101), (...",1359
1,Arnt,jaspar__MA0603.1,0.057200,3.073739,0.000799,1.000000,gene is annotated for similar motif transfac_p...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Rnf115', 1.1655901315529902), ('Klf6', 0.93...",471
2,Arnt,hocomoco__USF1_MOUSE.H11MO.0.A,0.066981,3.951676,0.000675,1.000000,gene is annotated for similar motif swissregul...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Rnf115', 1.1655901315529902), ('Map3k12', 0...",1118
3,Arnt,taipale_cyt_meth__ARNTL_RTCACGTGMN_eDBD,0.057509,3.101489,0.000479,0.389381,gene is orthologous to FBgn0264075 in D. melan...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Rnf115', 1.1655901315529902), ('Utp18', 0.5...",472
4,Arnt,cisbp__M4579,0.058929,3.228891,0.000998,1.000000,motif similar to swissregulon__mm__ARNT_ARNT2_...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Rnf115', 1.1655901315529902), ('Map3k12', 0...",1520
...,...,...,...,...,...,...,...,...,...,...
26481,Zfp729a,dbcorrdb__YY1__ENCSR000BNT_1__m1,0.062308,3.295964,0.000241,0.329135,motif similar to transfac_pro__M06259 ('V$ZNF9...,"frozenset({'activating', 'mm10__refseq-r80__50...","[('Hexim2', 0.3951984944559493), ('Celf1', 0.5...",1370
26482,Zfp729a,dbcorrdb__YY1__ENCSR000EWF_1__m1,0.061692,3.234288,0.000824,0.329135,motif similar to transfac_pro__M06259 ('V$ZNF9...,"frozenset({'activating', 'mm10__refseq-r80__50...","[('Hexim2', 0.3951984944559493), ('Celf1', 0.5...",1741
26483,Zfp729a,dbcorrdb__YY1__ENCSR000BNX_1__m1,0.060918,3.156835,0.000332,0.329135,motif similar to transfac_pro__M06259 ('V$ZNF9...,"frozenset({'activating', 'mm10__refseq-r80__50...","[('Hexim2', 0.3951984944559493), ('Celf1', 0.5...",1475
26484,Zfp729a,dbcorrdb__YY1__ENCSR000BKD_1__m1,0.061782,3.243253,0.000627,0.329135,motif similar to transfac_pro__M06259 ('V$ZNF9...,"frozenset({'activating', 'mm10__refseq-r80__50...","[('Hexim2', 0.3951984944559493), ('Celf1', 0.5...",1313


In [28]:
# ================== preprocess gene expression time series ==================

# compute z-scores
from scipy.stats import zscore
z_series = time_series.T.apply(zscore).T

# calculate distances of time-series 
ds = dtw.distance_matrix_fast( np.array(z_series), parallel=40, use_mp=True)

# ======================= preprocess TF target modules =======================
def get_target_genes(genestr):
    target = eval(genestr)
    target = [pair[0] for pair in target]
    return target

tf2target = {}
for tf, tg in zip(reg['TF'], reg['TargetGenes']):
    if tf not in tf2target.keys():
        tf2target[tf] = set(get_target_genes(tg))
    else:
        tf2target[tf].update(set(get_target_genes(tg)))

# ======================= preprocess TF target modules =======================
def build_multiplex_graph(time_series, dtw_dist, tf2target, k=20, MNN=True):
    multiplex_list = [] # the result list
    
    # keep the gene-index mapping
    genes = time_series.index
    g2i= {genes[i]:i for i in range(len(genes))}

    # build kNN graph of dtw dist
    sorted_index = np.argsort(dtw_dist, axis=1)
    kNN_src = sorted_index[:,0]
    kNN_dst = sorted_index[:,1:1+k]
    
    # use MNN to filter kNN
    if MNN:
        mNN_dst = []
        for src in kNN_src:
            mutual_nn = []
            for node in kNN_dst[src]:
                if src in kNN_dst[node]:
                    mutual_nn.append(node)
            mNN_dst.append(mutual_nn)       
        
    # the dtw layer
    g_dtw = ig.Graph()
    g_dtw.add_vertices(1+max(kNN_src))
    g_dtw.vs['name'] = genes
    edges_all=[]
    if MNN:
        kNN_dst = mNN_dst
    for src, dsts in zip(kNN_src, kNN_dst):
        for dst in dsts:
            n1=genes[src]; n1=g2i[n1]
            n2=genes[dst]; n2=g2i[n2]
            edges_all.append((n1,n2))
    g_dtw.add_edges(edges_all)        
    print('dtw layer n=',g_dtw.vcount(), 'e=',g_dtw.ecount())
    multiplex_list.append(g_dtw)

    # the scenic layer
    g_scenic = ig.Graph()
    g_scenic.add_vertices(1+max(kNN_src)) # same # of nodes!
    g_scenic.vs['name'] = genes
    edges_all=[]
    for tf, targets in tf2target.items():
        for g in targets:
            if g in genes and tf in genes:
                n1=tf; n1=g2i[n1];
                n2=g;  n2=g2i[n2];
                edges_all.append((n1,n2))
    g_scenic.add_edges(edges_all)
    print('scenic layer n=',g_scenic.vcount(), 'e=',g_scenic.ecount())
    multiplex_list.append(g_scenic)
    return multiplex_list

MG = build_multiplex_graph(time_series=time_series, dtw_dist=ds, tf2target=tf2target, k=5)


dtw layer n= 8191 e= 18876
scenic layer n= 8191 e= 18747


In [29]:
import pickle
with open('./FL-MG.pk','wb') as f:
    pickle.dump(MG, f)
    

# build multiplex network for Adult Bone Marrow group

In [32]:
# loading gene expression time series
ExprPath = "/home/csj/mywork/221107-path/221112-ABM-ILCprimed-PathExpr.csv"
            
time_series = pd.read_csv(ExprPath, index_col=0)

# loading scenic network modules
scenic_path='/home/csj/mywork/TSC/abm_regulons_seed1.csv' 
reg = pd.read_csv(scenic_path,
                  index_col=[0,1], header=[0,1])\
      .droplevel(0,axis=1).reset_index()


In [33]:
# ================== preprocess gene expression time series ==================
# quality control, filter out time series with low accumulations
time_series = time_series[time_series.sum(axis=1)>0.2]
time_series.to_csv("ABM-time_series.csv")

In [34]:
time_series

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
Mrpl15,0.059159,0.118925,0.259030,0.246470,0.269401,0.353712,0.366646,0.258772,0.332980,0.290683,...,0.166961,0.213423,0.232672,0.144111,0.374858,0.175033,0.139350,0.000000,0.074218,0.000000
Vcpip1,0.279726,0.098331,0.143871,0.192491,0.127961,0.114166,0.173672,0.124087,0.124087,0.124087,...,0.216201,0.000000,0.179112,0.183707,0.074191,0.236736,0.084750,0.196927,0.373418,0.083651
Snhg6,0.393931,0.482886,0.605918,0.538583,0.689325,0.712107,0.657839,0.494066,0.397838,0.259828,...,0.627726,0.685096,0.823154,0.618637,0.458502,0.469725,0.603797,0.273257,0.944928,0.846812
Cops5,0.539766,0.753910,0.580005,0.607494,0.651374,0.655363,0.790660,0.745598,0.799944,0.836641,...,0.439295,0.522984,0.247488,0.588978,0.489668,0.429354,0.439075,0.422482,0.399934,0.563430
Arfgef1,0.048052,0.111548,0.144883,0.086505,0.048620,0.096847,0.089825,0.110983,0.110983,0.110983,...,0.000000,0.154191,0.090055,0.235824,0.234583,0.151987,0.096334,0.121177,0.213999,0.155388
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uty,0.000000,0.000000,0.035886,0.035886,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.074503,0.144935,0.136562,0.000000,0.000000,0.000000,0.098815
Ddx3y,0.298873,0.176615,0.492623,0.419281,0.168488,0.175058,0.200350,0.178757,0.222592,0.222068,...,0.297693,0.117147,0.384977,0.623391,0.470065,0.255051,0.376683,0.439189,0.476922,0.530194
Kdm5d,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.090055,0.090055,0.000000,0.070432,0.070432,0.000000,0.000000,0.074218,0.082137
Eif2s3y,0.153645,0.430982,0.536633,0.500196,0.173472,0.130118,0.158011,0.313261,0.357097,0.356751,...,0.394275,0.081075,0.273316,0.536831,0.334355,0.282742,0.157142,0.296206,0.364452,0.344404


In [35]:
reg

,TF,MotifID,AUC,NES,MotifSimilarityQvalue,OrthologousIdentity,Annotation,Context,TargetGenes,RankAtMax
0,2610044O15Rik8,transfac_pro__M05990,0.067063,3.790296,0.000000,0.375000,gene is orthologous to ENSG00000198429 in H. s...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Akirin2', 0.3986951989968174), ('Synj2', 1....",599
1,Arid5b,swissregulon__hs__ARID5B.p2,0.065396,3.025479,0.000000,0.880471,gene is orthologous to ENSG00000150347 in H. s...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Pcsk1', 0.6212614932631535), ('Pparg', 0.79...",1543
2,Arid5b,cisbp__M3574,0.069565,3.405454,0.000000,0.880471,gene is orthologous to ENSG00000150347 in H. s...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Pparg', 0.7929489447089907), ('Flrt2', 0.40...",1241
3,Bach1,elemento__TGACGTCA,0.067441,3.570543,0.000367,0.072727,motif similar to transfac_pro__M07677 ('I$CNC_...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Crem', 0.5290666703928825), ('Zmynd15', 1.1...",3871
4,Bach1,transfac_pro__M04681,0.065483,3.393336,0.000139,0.072727,motif similar to transfac_pro__M07677 ('I$CNC_...,"frozenset({'activating', 'weight>75.0%', 'mm10...","[('Crem', 0.5290666703928825), ('Ncdn', 0.4428...",940
...,...,...,...,...,...,...,...,...,...,...
29649,Zfp386,transfac_pro__M06213,0.055064,3.060769,0.000000,0.378594,gene is orthologous to ENSG00000185947 in H. s...,"frozenset({'activating', 'top50perTarget', 'mm...","[('Eef2', 0.2648616930988497), ('Cul1', 0.1545...",2192
29650,Zfp467,factorbook__ZNF263,0.054521,3.316244,0.000208,0.843434,gene is orthologous to ENSG00000181444 in H. s...,"frozenset({'activating', 'top50perTarget', 'mm...","[('Ep300', 0.4811365072936285), ('Mctp1', 0.32...",2020
29651,Zfp467,cisbp__M4666,0.054674,3.336579,0.000066,0.843434,gene is orthologous to ENSG00000181444 in H. s...,"frozenset({'activating', 'top50perTarget', 'mm...","[('Ep300', 0.4811365072936285), ('Mctp1', 0.32...",3389
29652,Zfp712,cisbp__M4616,0.063236,3.133368,0.000069,0.215177,motif similar to flyfactorsurvey__crol-F7-16_S...,"frozenset({'activating', 'top50perTarget', 'mm...","[('Orc4', 0.0003455391599588), ('9930021J03Rik...",1937


In [36]:
# compute z-scores
from scipy.stats import zscore
z_series = time_series.T.apply(zscore).T

# calculate distances of time-series 
ds = dtw.distance_matrix_fast( np.array(z_series), parallel=40, use_mp=True)

# ======================= preprocess TF target modules =======================
def get_target_genes(genestr):
    target = eval(genestr)
    target = [pair[0] for pair in target]
    return target

tf2target = {}
for tf, tg in zip(reg['TF'], reg['TargetGenes']):
    if tf not in tf2target.keys():
        tf2target[tf] = set(get_target_genes(tg))
    else:
        tf2target[tf].update(set(get_target_genes(tg)))

# ======================= preprocess TF target modules =======================
def build_multiplex_graph(time_series, dtw_dist, tf2target, k=20, MNN=True):
    multiplex_list = [] # the result list
    
    # keep the gene-index mapping
    genes = time_series.index
    g2i= {genes[i]:i for i in range(len(genes))}

    # build kNN graph of dtw dist
    sorted_index = np.argsort(dtw_dist, axis=1)
    kNN_src = sorted_index[:,0]
    kNN_dst = sorted_index[:,1:1+k]
    
    # use MNN to filter kNN
    if MNN:
        mNN_dst = []
        for src in kNN_src:
            mutual_nn = []
            for node in kNN_dst[src]:
                if src in kNN_dst[node]:
                    mutual_nn.append(node)
            mNN_dst.append(mutual_nn)       
        
    # the dtw layer
    g_dtw = ig.Graph()
    g_dtw.add_vertices(1+max(kNN_src))
    g_dtw.vs['name'] = genes
    edges_all=[]
    if MNN:
        kNN_dst = mNN_dst
    for src, dsts in zip(kNN_src, kNN_dst):
        for dst in dsts:
            n1=genes[src]; n1=g2i[n1]
            n2=genes[dst]; n2=g2i[n2]
            edges_all.append((n1,n2))
    g_dtw.add_edges(edges_all)        
    print('dtw layer n=',g_dtw.vcount(), 'e=',g_dtw.ecount())
    multiplex_list.append(g_dtw)

    # the scenic layer
    g_scenic = ig.Graph()
    g_scenic.add_vertices(1+max(kNN_src)) # same # of nodes!
    g_scenic.vs['name'] = genes
    edges_all=[]
    for tf, targets in tf2target.items():
        for g in targets:
            if g in genes and tf in genes:
                n1=tf; n1=g2i[n1];
                n2=g;  n2=g2i[n2];
                edges_all.append((n1,n2))
    g_scenic.add_edges(edges_all)
    print('scenic layer n=',g_scenic.vcount(), 'e=',g_scenic.ecount())
    multiplex_list.append(g_scenic)
    return multiplex_list

MG = build_multiplex_graph(time_series=time_series, dtw_dist=ds, tf2target=tf2target, k=5)


dtw layer n= 10388 e= 21176
scenic layer n= 10388 e= 41660


In [37]:
import pickle
with open('./ABM-MG.pk','wb') as f:
    pickle.dump(MG, f)
    